<a href="https://colab.research.google.com/github/Munavvar-innovator/DG-SLAM-Evaluation/blob/main/hybrid_pose_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# =============================================================
# HYBRID POSE OPTIMIZATION - FINAL FIXED GRADIENT VERSION
# Gradients now work because rotation uses torch.stack()
# =============================================================

import os, numpy as np, torch, imageio.v2 as imageio, matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:",device)

# ====================== LOAD DATA ============================
root="/content/DG-SLAM/data/TUM/rgbd_dataset_freiburg3_long_office_household"
rgb_dir, depth_dir = root+"/rgb", root+"/depth"
rgb_files, depth_files = sorted(os.listdir(rgb_dir)), sorted(os.listdir(depth_dir))

def load_rgb(p): return imageio.imread(p).astype(np.float32)/255.0
def load_depth(p): return imageio.imread(p).astype(np.float32)/5000.0

fx,fy,cx,cy = 535.4,539.2,320.1,247.6

def depth_to_cloud(d):
    H,W = d.shape
    ys,xs = np.meshgrid(np.arange(H),np.arange(W),indexing="ij")
    X=(xs-cx)*d/fx; Y=(ys-cy)*d/fy; Z=d
    return np.stack([X,Y,Z],-1).reshape(-1,3)

# Pair 0→150
i1,i2=0,150
rgb1,depth1=load_rgb(rgb_dir+"/"+rgb_files[i1]), load_depth(depth_dir+"/"+depth_files[i1])
rgb2,depth2=load_rgb(rgb_dir+"/"+rgb_files[i2]), load_depth(depth_dir+"/"+depth_files[i2])

pts1=torch.tensor(depth_to_cloud(depth1),device=device)
pts2=torch.tensor(depth_to_cloud(depth2),device=device)

rgb1=torch.tensor(rgb1,device=device).permute(2,0,1)
rgb2=torch.tensor(rgb2,device=device).permute(2,0,1)


# =============== FIXED GRADIENT-SAFE ROTATION =================
def euler_to_R(rx,ry,rz):
    cx,cy,cz=torch.cos(rx),torch.cos(ry),torch.cos(rz)
    sx,sy,sz=torch.sin(rx),torch.sin(ry),torch.sin(rz)

    Rz=torch.stack([torch.stack([ cz,-sz,0.],0),
                    torch.stack([ sz, cz,0.],0),
                    torch.stack([0., 0.,1.],0)])

    Ry=torch.stack([torch.stack([ cy,0., sy],0),
                    torch.stack([0.,1.,0.],0),
                    torch.stack([-sy,0.,cy],0)])

    Rx=torch.stack([torch.stack([1.,0.,0.],0),
                    torch.stack([0.,cx,-sx],0),
                    torch.stack([0.,sx, cx],0)])

    return Rz@Ry@Rx   # retains gradient!

def se3(p):
    R=euler_to_R(p[0],p[1],p[2])
    t=p[3:6]
    return R,t


# ==================== HYBRID OPTIMISATION ====================
pose=torch.zeros(6,requires_grad=True,device=device)
opt=torch.optim.Adam([pose],lr=0.003)

loss_hist=[]; geo_hist=[]; photo_hist=[]

H,W=rgb1.shape[1:3]

for it in range(250):

    R,t=se3(pose)
    warp=(pts1@R.T)+t     # Nx3 diff-safe

    # geometric alignment
    geo=((warp-pts2)**2).mean()

    # project safely
    X,Y,Z=warp[:,0],warp[:,1],warp[:,2]+1e-6
    u=fx*(X/Z)+cx; v=fy*(Y/Z)+cy
    mask=(u>=0)&(u<W)&(v>=0)&(v<H)

    if mask.any():
        ui=u[mask].long(); vi=v[mask].long()
        photo=(rgb1[:,vi,ui]-rgb2[:,vi,ui]).abs().mean()
    else: photo=torch.tensor(0.,device=device)

    loss=0.5*geo+0.5*photo

    opt.zero_grad()
    loss.backward()
    opt.step()

    loss_hist.append(loss.item());geo_hist.append(geo.item());photo_hist.append(photo.item())

    if it%25==0: print(f"ITER {it} | loss {loss.item():.4f} | geo {geo.item():.4f} | photo {photo.item():.4f}")

print("\nFINAL OPTIMIZED POSE [rx ry rz tx ty tz]:\n",pose.detach().cpu().numpy())


# ===================== PLOTS ================================
plt.figure(figsize=(9,5))
plt.plot(loss_hist,label="Hybrid Loss");plt.grid();plt.legend();plt.title("Hybrid Pose Convergence (0→150)")
plt.show()

plt.figure(figsize=(9,5))
plt.plot(geo_hist,label="ICP Geometric Error")
plt.plot(photo_hist,label="Photometric Consistency")
plt.grid();plt.legend();plt.title("Loss Breakdown")
plt.show()


Running on: cpu


TypeError: expected Tensor as element 2 in argument 0, but got float